# LTL Synthesis with TensorFlow Transformer and Keras Transformer Trainer


In [ ]:
import tensorflow as tf

from ml2.datasets import CSVDataset, SplitDataset
from ml2.ltl.ltl_spec import DecompLTLSpec, LTLSpec, LTLSpecToSeqTokenizer
from ml2.ltl.ltl_syn import LTLSynSolution, LTLSynSolutionToSeqTokenizer
from ml2.train import KerasTransformerTrainer
from ml2.pipelines import TFTransformerPipeline


## Create Pipeline


### Create Input Tokenizer


In [ ]:
input_tokenizer = LTLSpecToSeqTokenizer(pad=256)


In [ ]:
spec_data = CSVDataset.load("scpa-2/val", "ltl-syn", dtype=DecompLTLSpec)
input_tokenizer.build_vocabulary(spec_data.generator())

### Create Target Tokenizer


In [ ]:
target_tokenizer = LTLSynSolutionToSeqTokenizer(eos=True, pad=128)


In [ ]:
target_data = CSVDataset.load("scpa-2/val", "ltl-syn", dtype=LTLSynSolution)
target_tokenizer.build_vocabulary(target_data.generator(), add_start=True)

### Create Pipeline


In [ ]:
model_config = {
    "alpha": 0.5,
    "beam_size": 2,
    "custom_pos_enc": True,
    "d_embed_dec": 128,
    "d_embed_enc": 128,
    "d_ff": 512,
    "dropout": 0.0,
    "dtype_float": tf.float32,
    "dtype_int": tf.int32,
    "ff_activation": "relu",
    "num_heads": 4,
    "num_layers_dec": 4,
    "num_layers_enc": 4,
}


In [ ]:
pipeline = TFTransformerPipeline(
    name="t-0",
    project="ltl-syn",
    model_config=model_config,
    input_tokenizer=input_tokenizer,
    target_tokenizer=target_tokenizer,
    max_input_length=256,
    max_target_length=128,
)


## Create Trainer


In [ ]:
data = SplitDataset.load("scpa-2", "ltl-syn")

In [ ]:
trainer = KerasTransformerTrainer(
    pipeline=pipeline,
    train_dataset=data["train"],
    val_dataset=data["val"],
    steps=10
)

In [ ]:
trainer.train()